## Prueba de mejora de rag con late chunking

In [ ]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
import nltk
from dotenv import load_dotenv
from PyPDF2 import PdfReader

# ------------------------------
# 1. CONFIGURACIÓN INICIAL
# ------------------------------

# Cargar variables de entorno (asegúrate de tener un archivo .env con OPENAI_API_KEY)
load_dotenv()

# Descargar los recursos necesarios de NLTK para tokenización
nltk.download('punkt')

# Configurar la API key de OpenAI
openai.api_key = os.getenv('OPENAI_API_KEY')

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# ------------------------------
# 2. FUNCIÓN PARA CARGAR PDFs
# ------------------------------

def load_pdf_texts(root_folder):
    """
    Recorre recursivamente el directorio 'root_folder' y extrae el texto de cada PDF.
    Se agrega metadata (nombre del bloque y del archivo) al inicio del texto.
    
    Retorna:
      - documents: lista de textos de cada PDF (con metadata incluida).
      - metadata: lista de diccionarios con información del bloque, nombre del archivo y ruta.
    """
    documents = []
    metadata = []
    
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                file_path = os.path.join(subdir, file)
                try:
                    reader = PdfReader(file_path)
                    text = ""
                    for page in reader.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text + "\n"
                    # Suponemos que el nombre del bloque es el nombre del directorio inmediato donde se encuentra el PDF
                    block_name = os.path.basename(subdir)
                    # Incluir metadata en el texto para que el LLM sepa el origen
                    document_text = f"[Bloque: {block_name} | PDF: {file}]\n{text}"
                    documents.append(document_text)
                    metadata.append({"block": block_name, "file": file, "path": file_path})
                except Exception as e:
                    print(f"Error al leer {file_path}: {e}")
    return documents, metadata

# Ruta de la carpeta que contiene los bloques (cambia esta ruta si es necesario)
pdf_root = "temario_opos"
documents, doc_metadata = load_pdf_texts(pdf_root)
print(f"Se han cargado {len(documents)} documentos desde '{pdf_root}'.")

# ------------------------------
# 3. CREACIÓN DEL ÍNDICE FAISS
# ------------------------------

# Generar embeddings para cada documento
document_embeddings = embedding_model.encode(documents, convert_to_numpy=True)

# Crear el índice FAISS usando la distancia L2
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

# ------------------------------
# 4. FUNCIÓN DE LATE CHUNKING
# ------------------------------

def late_chunking(text, max_chunk_tokens=300, overlap_tokens=50):
    """
    Fragmenta el texto en chunks de hasta 'max_chunk_tokens' tokens,
    agregando un solapamiento de 'overlap_tokens' entre fragmentos.
    
    Se utiliza el tokenizador de oraciones de NLTK.
    """
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = sentence.split()
        num_tokens = len(sentence_tokens)
        
        # Si al agregar la oración se supera el límite, se crea un nuevo chunk
        if current_tokens + num_tokens > max_chunk_tokens:
            chunks.append(current_chunk.strip())
            # Solapamiento: tomar los últimos 'overlap_tokens' tokens del chunk anterior
            current_chunk_tokens = current_chunk.split()
            current_chunk = " ".join(current_chunk_tokens[-overlap_tokens:]) if overlap_tokens > 0 else ""
            current_tokens = len(current_chunk.split())
        
        current_chunk += " " + sentence
        current_tokens += num_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# ------------------------------
# 5. FUNCIÓN PARA RESPONDER CONSULTAS
# ------------------------------

def answer_query(query, top_k=4):
    """
    Procesa la consulta del usuario:
      - Genera el embedding de la consulta.
      - Recupera los top_k documentos (o sus fragmentos) más relevantes usando FAISS.
      - Si el documento es muy largo, se aplica late chunking para fragmentarlo.
      - Se prepara el prompt combinando el contexto y la pregunta.
      - Se llama a la API de ChatCompletion para obtener la respuesta.
    """
    # Generar embedding para la consulta
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    
    # Buscar los top_k documentos más relevantes
    distances, indices = index.search(query_embedding, top_k)
    
    retrieved_docs = []
    for i in indices[0]:
        # Recuperar el documento y su metadata para incluir en el contexto
        doc_text = documents[i]
        meta = doc_metadata[i]
        retrieved_docs.append((doc_text, meta))
    
    # Procesar cada documento recuperado
    context_chunks = []
    for doc_text, meta in retrieved_docs:
        # Si el documento es muy largo, se fragmenta para no sobrepasar el límite de tokens
        if len(doc_text.split()) > 300:
            chunks = late_chunking(doc_text, max_chunk_tokens=800, overlap_tokens=150)
            context_chunks.extend(chunks)
        else:
            context_chunks.append(doc_text)
    
    # Combinar los fragmentos en un único contexto
    context = "\n\n".join(context_chunks)
    
    # Preparar el prompt para el modelo de lenguaje
    prompt = f"Contexto:\n{context}\n\nPregunta: {query}\nRespuesta:"
    
    # Llamar a la API de ChatCompletion (gpt-4o-mini)
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Eres un asistente educativo experto. Responde basándote únicamente en el contexto proporcionado."},
            {"role": "user", "content": prompt}
        ],
        #max_tokens=200,
        temperature=0.0,
    )
    
    return response.choices[0].message.content.strip()

# ------------------------------
# 6. EJEMPLO DE USO
# ------------------------------

if __name__ == "__main__":
    consulta = input("Introduce tu consulta: ")
    respuesta = answer_query(consulta)
    print("\nRespuesta del chatbot:")
    print(respuesta)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joseluis.fernandez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Se han cargado 67 documentos desde 'temario_opos'.


RateLimitError: Request too large for gpt-4o in organization org-coN5zOQyLyZPTark25qT6s3T on tokens per min (TPM): Limit 30000, Requested 55444. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.